# Extracción de datos de la API del BCRP.
La Base de Datos de Estadísticas del BCRP (BCRPData) proporciona la herramienta API para Desarrolladores como una de las facilidades que se brindan para la obtención y uso de las series macroeconómicas. (https://estadisticas.bcrp.gob.pe/estadisticas/series/documentos/bcrpdataapi.pdf)

Este código costruira una URL que se utilizará para acceder a datos de estadísticas proporcionados por el Banco Central de Reserva del Perú (BCRP) a través de su API. 

In [13]:
url_base="https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
cod_ser="PN38063GM" # Tasa de Desempleo (%)
formato="/csv"
per="/2018-9/2023-9"
url=url_base+cod_ser+formato+per
print(url)
#args="address=1600+Amphitheatre+Parkway, +Mountain+View, +CA&"+key_api
#No necesitamos poner la url en formato de diccionario, no hay "?" no usamos query con argumentos con key y values

https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PN38063GM/csv/2018-9/2023-9


Utilizando la biblioteca requests de Python para realizar una solicitud HTTP GET a la URL construida previamente

In [14]:
import requests 
response = requests.get(url)  # Response 200 todo OK 
print(response)

<Response [200]>


Después de la solicitud HTTP GET y procesamos la respuesta obtenida de la API del BCRP y comocamos los datos en un DF.

In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Comprueba si la solicitud se realizó con éxito (código de respuesta 200)
if response.status_code == 200:
    # Utiliza BeautifulSoup para eliminar el HTML y obtener el texto
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.get_text()
    
    # Utiliza una expresión regular para extraer la fecha y el valor
    pattern = r'"(.*?)","(.*?)"'
    matches = re.findall(pattern, data)
    
    # Crea una lista de diccionarios para los datos
    data_list = []
    for match in matches:
        date = match[0].replace('"', '')  # Elimina las comillas
        value = float(match[1])
        data_list.append({'Fecha': date, 'Tasa de Desempleo (%)': value})
    
    # Crea un DataFrame a partir de la lista de diccionarios
    df = pd.DataFrame(data_list)
    
    # Imprime el DataFrame
    print(df)
else:
    print(f"La solicitud GET a la API falló con el código de respuesta: {response.status_code}")

       Fecha  Tasa de Desempleo (%)
0   Sep.2018               6.181888
1   Oct.2018               6.425933
2   Nov.2018               6.247634
3   Dic.2018               6.126948
4   Ene.2019               6.476252
..       ...                    ...
56  May.2023               6.778779
57  Jun.2023               6.551520
58  Jul.2023               6.264338
59  Ago.2023               6.637680
60  Sep.2023               6.731492

[61 rows x 2 columns]


In [24]:
# Mapear los nombres de los meses a números de mes
meses = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

# Función para convertir las fechas
def convertir_fecha(fecha):
    partes = fecha.split('.')
    mes = meses.get(partes[0], '01')  # Si el mes no se encuentra, se usa '01' como valor por defecto
    nueva_fecha = f'01/{mes}/{partes[1]}'
    return nueva_fecha

# Aplicar la función a la columna 'Fecha'
df['Fecha'] = df['Fecha'].apply(convertir_fecha)

# Mostrar el DataFrame resultante
print(df)

         Fecha  Tasa de Desempleo (%)
0   01/09/2018               6.181888
1   01/10/2018               6.425933
2   01/11/2018               6.247634
3   01/12/2018               6.126948
4   01/01/2019               6.476252
..         ...                    ...
56  01/05/2023               6.778779
57  01/06/2023               6.551520
58  01/07/2023               6.264338
59  01/08/2023               6.637680
60  01/09/2023               6.731492

[61 rows x 2 columns]
